In [5]:
word2vec_model = '/Users/iijima.s.ad/git/JASen/word2vec_100.txt'
comment_folder = '/Users/iijima.s.ad/git/article-extractor/articles/fox/drones/*'
comment_files = '/Users/iijima.s.ad/git/article-extractor/articles/fox/drones/*/comments.json'
output_folder = '/Users/iijima.s.ad/git/workspace/articles/fox/drones/'

origin_articles = '/Users/iijima.s.ad/git/workspace/text/data/*'
ner_article = '/Users/iijima.s.ad/git/workspace/text/NER_article2.txt'
ner_insert_article = '/Users/iijima.s.ad/git/workspace/text/NER_insert_article.txt'

In [2]:
import en_core_web_sm
import spacy

nlp = spacy.load('en_core_web_sm', exclude=['tok2vec', 'parser'])

/Users/iijima.s.ad/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
use_replies = False
accept_labels = set(nlp.get_pipe('ner').labels)
# accept_labels = {'PERSON'}
# accept_labels = set()

In [4]:
import re

p = re.compile('\s+')
def trim(sentence: str):
    return p.sub(' ', sentence)

In [101]:
import glob
import json
import os
import random

test_paths = set()

for path in random.sample(glob.glob(comment_folder), k=50):
    with open(os.path.join(path, 'info.json'), 'r') as f:
        comment_count = float(json.load(f)['comment_count'])
        if comment_count >= 10 and len(test_paths) < 10:
            test_paths.add(path + 'comment.json')

In [ ]:


import glob
import json
import os

for folder_path in glob.glob(comment_folder):
    sentence_list = []

    if not os.path.exists(os.path.join(folder_path, 'comments.json')):
        continue

    with open(os.path.join(folder_path, 'comments.json'), 'r') as comment_json:
        comment_stack: list = json.load(comment_json)
    
    while len(comment_stack) > 0:
        comment = comment_stack.pop()

        if 'context' not in comment:
            continue

        if 'replies' in comment and use_replies:
            comment_stack += comment['replies']

        doc = nlp(trim(comment['context']))
        tokens = list(doc)
        ents_dict = {ent.start: ent for ent in doc.ents}
        words = []

        i = 0
        while i < len(tokens):
            if i in ents_dict and ents_dict[i].label_ in accept_labels:
                ent = ents_dict[i]
                words.append(f'[[{ent.label_}]]')
                i = ent.end
            else:
                words.append(tokens[i].text.lower())
                i += 1
        
        sentence_list.append(' '.join(words))

    file_base = 'NER_comments_with_replies.txt' if use_replies else 'NER_only_root.txt'
    with open(os.path.join(folder_path, file_base), 'w') as f:
        f.writelines([sentence + '\n' for sentence in sentence_list])


In [7]:
# News Article Corpus をNERにかける
import datetime
import glob
import json
import os


count = 0

with open(ner_article, 'w') as output_file, open(ner_insert_article, 'w') as output_file2:

    article_paths_list = [[]]
    for article_path in glob.glob(origin_articles):
        article_paths_list[-1].append(article_path)
        if len(article_paths_list[-1]) % 1000 == 0:
            article_paths_list.append([])


    for article_paths in article_paths_list:

        sentences = []

        for article_path in article_paths:
            with open(article_path, 'r') as f:
                sentences += [line.strip() for line in f.readlines() if line != '\n']

        for doc in nlp.pipe(sentences, batch_size=2000):
            tokens = list(doc)
            ents_dict = {ent.start: ent for ent in doc.ents}
            words, words2 = [], []

            i = 0
            while i < len(tokens):
                if i in ents_dict and ents_dict[i].label_ in accept_labels:
                    ent = ents_dict[i]
                    words.append(f'[[{ent.label_}]]')
                    words2.append(f'[[{ent.label_}]]')
                    words2.append(ent.text.lower())
                    i = ent.end
                else:
                    words.append(tokens[i].lemma_.lower())
                    words2.append(tokens[i].lemma_.lower())
                    i += 1
            
            if len(words) > 0:
                output_file.write(' '.join(words) + '\n')
                output_file2.write(' '.join(words2) + '\n')
            
            count += 1
            if count % 100000 == 0:
                print(f'done {count}, time: {datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")}')


done 100000, time: 2022/10/14 02:16:02
done 200000, time: 2022/10/14 02:24:30
done 300000, time: 2022/10/14 02:32:38
done 400000, time: 2022/10/14 02:41:18
done 500000, time: 2022/10/14 02:49:27
done 600000, time: 2022/10/14 02:58:12
done 700000, time: 2022/10/14 03:06:39
done 800000, time: 2022/10/14 03:15:04
done 900000, time: 2022/10/14 03:23:42
done 1000000, time: 2022/10/14 03:31:54
done 1100000, time: 2022/10/14 03:39:48
done 1200000, time: 2022/10/14 03:47:58
done 1300000, time: 2022/10/14 03:56:29
done 1400000, time: 2022/10/14 04:05:03
done 1500000, time: 2022/10/14 04:13:35
done 1600000, time: 2022/10/14 04:22:01
done 1700000, time: 2022/10/14 04:30:21
done 1800000, time: 2022/10/14 04:38:45
done 1900000, time: 2022/10/14 04:47:08
done 2000000, time: 2022/10/14 04:55:55
done 2100000, time: 2022/10/14 05:04:07
done 2200000, time: 2022/10/14 05:12:42
done 2300000, time: 2022/10/14 05:20:40
done 2400000, time: 2022/10/14 05:29:28
done 2500000, time: 2022/10/14 05:37:56
done 2600

In [4]:
# 記事のコメント数を取得する
import glob
import json
import os

url_set = set()
count, article_count = 0, 0

for dir_path in glob.glob('/Users/iijima.s.ad/git/article-extractor/articles/fox/*/*/'):
    if not os.path.exists(os.path.join(dir_path, 'info.json')):
        continue

    with open(os.path.join(dir_path, 'info.json'), 'r') as f:
        info = json.load(f)
        url = info['URL']
        
    if url in url_set:
        continue
    url_set.add(url)

    with open(os.path.join(dir_path, 'comments.json'), 'r') as f:
        stack = [a for a in json.load(f)]

    comment_count = 0
    while len(stack) > 0:
        comment_count += 1
        comment = stack.pop()
        if 'replies' in comment:
            stack += comment['replies']

    # if comment_count > 10:
    article_count += 1
    count += comment_count

print(count)

355973


In [7]:
# Article.txtをNERにかける
import datetime
import glob
import json
import os


for path in glob.glob('/Users/iijima.s.ad/git/article-extractor/articles/fox/*/*/article.txt'):
    with open(path, 'r') as f:
        sentences = [line.strip() for line in f.readlines() if line.strip() != '']

    article_ner_file = path.replace('article.txt', 'article_ner.txt')
    article_joint_file = path.replace('article.txt', 'article_joint.txt')
    with open(article_ner_file, 'w') as output_file, open(article_joint_file, 'w') as output_file2:
        for doc in nlp.pipe(sentences, batch_size=2000):
            tokens = list(doc)
            ents_dict = {ent.start: ent for ent in doc.ents}
            words, words2 = [], []

            i = 0
            while i < len(tokens):
                if i in ents_dict and ents_dict[i].label_ in accept_labels:
                    ent = ents_dict[i]
                    words.append(f'[[{ent.label_}]]')
                    words2.append(f'[[{ent.label_}:::{ent.text.lower()}]]')
                    i = ent.end
                else:
                    words.append(tokens[i].lemma_.lower())
                    words2.append(tokens[i].lemma_.lower())
                    i += 1
            
            if len(words) > 0:
                output_file.write(' '.join(words) + '\n')
                output_file2.write(' '.join(words2) + '\n')